In [ ]:
# ¡IMPORTANTE! Leer primero el apartado de requisitos para descargar todo lo necesario para ejecutar el programa

# Importamos todas las librerías necesarias
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import selenium 
from selenium import webdriver
from selenium.webdriver.support.ui import Select

# Pasamos como argumento la web pricipal que scrapeamos
url = "https://es.investing.com/equities/most-active-stocks"

# Y la ruta en la que hemos guardado la extensión previamente descargada "ChromeDriver"
PATH = "/Applications/chromedriver"

def most_active_stocks(url, PATH):
     
    #Para descargar el contenido de la pagina
    
    #url = "https://es.investing.com/equities/"stock"-historical-data"
    
    r=requests.get(url)

    #Creamos el árbol de objetos Python que representan al documento HTML (la sopa)
    soup = BeautifulSoup(r.text, 'lxml')
    
    #Tenemos que "engañar" a la pagina, para que crea que somos un navegador, por ello, incluimos headers, sino, obtendriamos el siguiente error:
    #<title>403 You are banned from this site.  Please contact via a different client configuration if you believe that this is a mistake.</title>

    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36'}
    page=requests.get(url,headers=header)

    # Creamos las listas y el diccionario para recoger el data set
    list_name = []
    list_last = []
    list_high = []
    list_vol = []
    list_low = []
    list_time = []
    list_var = []
    list_por_var = []
    list_web = []
    dict_data = {}
    
    # Comprobamos que la petición nos devuelve un Status Code = 200
    status_code = page.status_code
    if status_code == 200:
        
        soup=BeautifulSoup(page.content,'html.parser')
        result = soup.find('td',attrs={'class':'left bold plusIconTd elp'})
    
        # Crearemos un bucle que vaya recogiendo los datos y los vaya metiendo a sus correspondientes listas
        for link in soup.find_all('td',attrs={'class':'left bold plusIconTd elp'}):
            name= link.get_text()
            list_name.append(name)
            
            web_name = re.findall('"/equities/(.*)" title',str(link))[0]
            list_web.append(web_name)
            
            line = link.find('span',{'class':'alertBellGrayPlus js-plus-icon genToolTip oneliner'})
            id_valor = re.findall('\D(\d{3,7})\D', str(line))[0]
            
            
            new_class_1 = "align_right pid-"+ id_valor +"-last"
            last=soup.find('td',attrs={'class':new_class_1}).getText()
            list_last.append(last)

            new_class_2 = "pid-"+ id_valor +"-time"
            time=soup.find('td',attrs={'class':new_class_2}).getText()
            list_time.append(time)

            new_class_3 = "pid-"+ id_valor +"-turnover"
            vol=soup.find('td',attrs={'class':new_class_3}).getText()
            list_vol.append(vol)

            new_class_4 = "align_right pid-"+ id_valor+"-high"
            high=soup.find('td',attrs={'class':new_class_4}).getText()
            list_high.append(high)

            new_class_5 = "pid-"+ id_valor +"-low"
            low=soup.find('td',attrs={'class':new_class_5}).getText()
            list_low.append(low)

            if soup.find('td',attrs={'class':"bold redFont pid-"+id_valor+"-pc"}) != None:
                var = soup.find('td',attrs={'class':"bold redFont pid-"+id_valor+"-pc"}).getText()
                por_var = soup.find('td',attrs={'class':"bold redFont pid-"+id_valor+"-pcp"}).getText()

            elif soup.find('td',attrs={'class':"bold greenFont pid-"+id_valor+"-pc"}) != None:
                var = soup.find('td',attrs={'class':"bold greenFont pid-"+id_valor+"-pc"}).getText()
                por_var = soup.find('td',attrs={'class':"bold greenFont pid-"+id_valor+"-pcp"}).getText()

            else:
                var = soup.find('td',attrs={'class':"bold blackFont pid-"+id_valor+"-pc"}).getText()
                por_var = soup.find('td',attrs={'class':"bold blackFont pid-"+id_valor+"-pcp"}).getText()

            list_var.append(var)
            list_por_var.append(por_var)
            
            
        for web_name in list_web:
            #Para descargar el contenido de la pagina
            
            if web_name == "intl.-cons.-air-grp?cid=13809":
                url_2 = "https://es.investing.com/equities/intl.-cons.-air-grp-historical-data?cid=13809"
            elif web_name == "arcelormittal-reg?cid=32439":
                url_2 = "https://es.investing.com/equities/arcelormittal-reg-historical-data?cid=32439"
            elif web_name == "eads?cid=32302":
                url_2 = "https://es.investing.com/equities/eads-historical-data?cid=32302"
            else:
                url_2 = "https://es.investing.com/equities/" + web_name + "-historical-data"
    
            r=requests.get(url_2)

            #Creamos el árbol de objetos Python que representan al documento HTML (la sopa)
            soup = BeautifulSoup(r.text, 'lxml')

            #Tenemos que "engañar" a la pagina, para que crea que somos un navegador, por ello, incluimos headers, sino, obtendriamos el siguiente error:
            #<title>403 You are banned from this site.  Please contact via a different client configuration if you believe that this is a mistake.</title>

            header={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36'}
            page=requests.get(url_2,headers=header)
            
            # Comprobamos que la petición nos devuelve un Status Code = 200
            status_code = page.status_code
            if status_code == 200:

                # Se selciona la extension descargada y con ella se ejecuta atomáticamente nuestra  búsqueda
                driver = webdriver.Chrome(PATH)
                driver.get(url_2)
                
                # Elegimos el desplgable y la opción que queremos mostrar
                select = Select(driver.find_element_by_id('data_interval'))
                select.select_by_visible_text('Mensual')
                
                soup=BeautifulSoup(page.content,'html.parser')
                
                # Creamos las listas y el diccionario para recoger el data set
                list_date_hist = []
                list_price_hist = []
                list_open_hist = []
                list_high_hist = []
                list_low_hist = []
                list_vol_hist = []
                list_var_hist = []
                dict_data_hist = {}
                
                # Crearemos un bucle que vaya recogiendo los datos y los vaya metiendo a sus correspondientes listas
                for dato in driver.find_elements_by_xpath('//*[@id="curr_table"]'):
                    data_equities = dato.text
                    
                    # Se recoge toda la información en un mismo elemento que debemos ir dividiendo y clasificando en las listas
                    data_by_month = data_equities.split('\n')
                    
                    indice = 1
                    conuter_year = 0
                    
                    for i in data_by_month: 
                        data_by_value = data_by_month[indice].split()
                        indice += 1
                        
                        if indice == len(data_by_month):
                            break
                        
                        counter = 0    
                        
                        for j in data_by_value:  
                            if counter == 0: 
                                if conuter_year < 11:
                                    list_date_hist.append(j + " 2020")
                                else:
                                    list_date_hist.append(j + " 2019")
                            elif counter == 2:
                                list_price_hist.append(j)
                            elif counter == 3:
                                list_open_hist.append(j)
                            elif counter == 4:
                                list_high_hist.append(j)
                            elif counter == 5:
                                list_low_hist.append(j)
                            elif counter == 6:
                                list_vol_hist.append(j)
                            elif counter == 7:
                                list_var_hist.append(j)                          
                            counter += 1 
                            
                            if counter == len(data_by_value):
                                break
                        
                        conuter_year += 1
                
                # Creamos las entreadas del diccionario y lo convertimos en un dataframe    
                dict_data_hist["Fecha"] = list_date_hist
                dict_data_hist["Último valor"] = list_price_hist
                dict_data_hist["Valor apertura"] = list_open_hist
                dict_data_hist["Máx. valor"] = list_high_hist
                dict_data_hist["Min. valor"] = list_low_hist
                dict_data_hist["Volumen"] = list_vol_hist
                dict_data_hist["% Variación"] = list_var_hist
                
                df_hist = pd.DataFrame(dict_data_hist, columns = ['Fecha', 'Último valor', 'Valor apertura', 'Máx. valor', 'Min. valor', 'Volumen', '% Variación'])

                # Descargamos un archivo .csv 
                final_datafile_hist = df_hist.to_csv('WebScraping_SP_stock_'+ web_name + '.csv', sep=',')
                    
            else:
                print("Status Code %d" % status_code)
    
        # Creamos las entreadas del diccionario y lo convertimos en un dataframe
        dict_data['Nombre'] = list_name
        dict_data['Último valor'] = list_last
        dict_data['Máx. valor'] = list_high
        dict_data['Min. valor'] = list_low
        dict_data['Variación'] = list_var
        dict_data['% Variación'] = list_por_var
        dict_data['Volumen'] = list_vol
        dict_data['Hora'] = list_time

        df = pd.DataFrame(dict_data, columns = ['Nombre', 'Último valor', 'Máx. valor', 'Min. valor', 'Variación', '% Variación', 'Volumen', 'Hora'])

        # Descargamos un archivo .csv 
        final_datafile = df.to_csv('WebScraping_SP_stocks.csv', sep=',')
        
        
    else:
        print("Status Code %d" % status_code)
        
    return

# Ejecutamos la función creada

most_active_stocks(url, PATH)
